In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


train = pd.read_csv("../input/train.csv")
test = pd.read_csv('../input/test.csv')
train

In [ ]:
# train.info()
# train.describe()
# train.describe(include=['O'])

In [ ]:
y_train = train['Survived']

In [ ]:
train.drop(['Survived'], axis=1, inplace=True)

# survive_rate = len(y[y==1])/len(y)  # survive rate is 38.38%
# train['Cabin'] = pd.factorize(train['Cabin'])[0]
train.drop(['Cabin'], axis=1, inplace=True)
train['Embarked'].fillna('S', inplace=True)  # fill nan in Embarked with top freq 'S'
train['Embarked'] = pd.factorize(train['Embarked'])[0]
train['Ticket'] = pd.factorize(train['Ticket'])[0]
# train.drop(['Ticket'], axis=1, inplace=True)

train['Sex'] = pd.factorize(train['Sex'])[0]

# the title in the Name may highly correlated with Age Sex, so drop the Name col
# now all the categorical data has been convert to numerical data
train.drop(['PassengerId'], axis=1, inplace=True)
X_train = train

In [ ]:

X_train['Title'] = X_train['Name'].str.extract('([A-Za-z]+)\.', expand=False)
X_train['Title'] = X_train['Title'].replace('Mlle', 'Miss')
X_train['Title'] = X_train['Title'].replace('Ms', 'Miss')
X_train['Title'] = X_train['Title'].replace('Mme', 'Mrs')

X_train['Title'] = X_train['Title'].replace('Rev', 'Mr')

X_train['Title'] = X_train['Title'].replace(['Capt', 'Col', 'Countess', 'Don', 'Dr', 'Major', 'Sir', 'Jonkheer', 'Lady'], 'Noble')
# X_train.info()

In [ ]:
X_age = X_train.dropna(axis=0)
X_age_class = X_age.groupby('Pclass').agg({'Age': np.average})
X_age_title = X_age.groupby('Title').agg({'Age': np.average})
print(X_age.groupby('Pclass').agg({'Age': np.average}))
print(X_age.groupby('Title').agg({'Age': np.average}))

In [ ]:
X_age_title.loc['Master']

In [ ]:
X_train['Age'].fillna(0, inplace=True)
X_train.drop('Name', axis=1, inplace=True)


In [ ]:
X_train_M_age = X_train[X_train['Age'] == 0]
X_train_M_age
# for cla in X_train_M_age['Pclass']:
#     for title in X_train_M_age['Title']:
#         X_train_M_age[(X_train_M_age['Pclass'] == cla) & (X_train_M_age['Title'] == title)]['Age'] = (X_age_class.loc[cla] + X_age_title.loc[title]) / 2
# X_train_M_age

In [ ]:
# fill missing ages
# ------------------------------------
train = pd.read_csv("../input/train.csv")

y = train['Survived']

train.drop(['Survived'], axis=1, inplace=True)

train.drop(['Cabin'], axis=1, inplace=True)
train['Embarked'].fillna('S', inplace=True)  # fill nan in Embarked with top freq 'S'
train['Embarked'] = pd.factorize(train['Embarked'])[0]
train['Ticket'] = pd.factorize(train['Ticket'])[0]
train['Sex'] = pd.factorize(train['Sex'])[0]
train.drop(['PassengerId'], axis=1, inplace=True)

train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=False)
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
train['Title'] = train['Title'].replace('Rev', 'Mr')

train['Title'] = train['Title'].replace(['Capt', 'Col', 'Countess', 'Don',
                                         'Dr', 'Major', 'Sir', 'Jonkheer', 'Lady'], 'Noble')

X_age = train.dropna(axis=0)
X_age_class = X_age.groupby('Pclass').agg({'Age': np.average})
X_age_title = X_age.groupby('Title').agg({'Age': np.average})
print(X_age.groupby('Pclass').agg({'Age': np.average}))
print(X_age.groupby('Title').agg({'Age': np.average}))

train['Age'].fillna(0, inplace=True)
train.drop('Name', axis=1, inplace=True)

X_train_M_age = train[train['Age'] == 0]

X_train_M_age.set_value(X_train_M_age[X_train_M_age['Title'] == 'Master'].index,
                        'Age', X_age_title.loc['Master'].item())
X_train_M_age.set_value(X_train_M_age[X_train_M_age['Title'] == 'Noble'].index,
                        'Age', X_age_title.loc['Noble'].item())

for cla in X_train_M_age['Pclass']:
    for title in ['Miss', 'Mr', 'Mrs']:
        X_train_M_age.set_value(X_train_M_age[(X_train_M_age['Pclass'] == cla) & (X_train_M_age['Title'] == title)].index, 'Age',
         (X_age_class.loc[cla].item() + X_age_title.loc[title].item()) / 2)
        
train.set_value(train[train['Age'] == 0].index, 'Age', X_train_M_age['Age'].values)  # set missing ages completed

train['Title'] = pd.factorize(train['Title'])[0] + 1
train['Embarked'] = train['Embarked'] + 1
train['Ticket'] = train['Ticket'] + 1
train['Sex'] = train['Sex'] + 1


X_train = train.drop('Ticket', axis=1)

In [ ]:
test = pd.read_csv('../input/test.csv')
# test
test.info()  # notice there is additional Null in fare of test
test['Title'] = test['Name'].str.extract('([A-Za-z]+)\.', expand=False)
X_age = test.dropna(axis=0)
X_age_title = X_age.groupby('Title').agg({'Age': np.average})
X_age_title

In [ ]:
test = pd.read_csv('../input/test.csv')
test.drop(['Cabin'], axis=1, inplace=True)
test.fillna(test['Fare'].mean(), inplace=True)
test['Embarked'] = pd.factorize(test['Embarked'])[0]
test['Sex'] = pd.factorize(test['Sex'])[0]
test.drop(['PassengerId'], axis=1, inplace=True)

test['Title'] = test['Name'].str.extract('([A-Za-z]+)\.', expand=False)

test['Title'] = test['Title'].replace(['Col', 'Dr', 'Dona'], 'Noble')

X_age = test.dropna(axis=0)
X_age_class = X_age.groupby('Pclass').agg({'Age': np.average})
X_age_title = X_age.groupby('Title').agg({'Age': np.average})

test['Age'].fillna(0, inplace=True)
test.drop('Name', axis=1, inplace=True)

X_test_M_age = test[test['Age'] == 0]

X_test_M_age.set_value(X_test_M_age[X_test_M_age['Title'] == 'Master'].index,
                        'Age', X_age_title.loc['Master'].item())
X_test_M_age.set_value(X_test_M_age[X_test_M_age['Title'] == 'Noble'].index,
                        'Age', X_age_title.loc['Noble'].item())

for cla in X_test_M_age['Pclass']:
    for title in ['Miss', 'Mr', 'Mrs']:
        X_test_M_age.set_value(X_test_M_age[(X_test_M_age['Pclass'] == cla) & (X_test_M_age['Title'] == title)].index, 'Age',
         (X_age_class.loc[cla].item() + X_age_title.loc[title].item()) / 2)
        
test.set_value(test[test['Age'] == 0].index, 'Age', X_test_M_age['Age'].values)  # set missing ages completed

test['Title'] = pd.factorize(test['Title'])[0] + 1
test['Embarked'] = test['Embarked'] + 1
test['Sex'] = test['Sex'] + 1

X_test = test.drop('Ticket', axis=1)

X_test.info()
# X_test

In [ ]:
# Important for logistic regression, svm, NN
# important to introduce the normalization to deal with verious feature scales
X_train_scaled = MinMaxScaler().fit_transform(X_train)
X_test_scaled = MinMaxScaler().fit_transform(X_test)


In [ ]:
# logistic regression
# For better fit, bigger C values specify weaker regularization
lr = LogisticRegression(C=10).fit(X_train_scaled, y_train)
train_score = lr.score(X_train_scaled, y_train)
train_score

In [ ]:
# see the output requirement
output = pd.read_csv('../input/gender_submission.csv')
output

In [ ]:
y_pre_lr = lr.predict(X_test_scaled)
out_lr = pd.DataFrame({'PassengerId': test['PassengerId'].values, 'Survived': y_pre_lr})
#out_lr

In [ ]:
# SVM
svm = SVC(gamma=1, C=250).fit(X_train_scaled, y_train)
train_score = svm.score(X_train_scaled, y_train)
train_score

# train_score = []
# for this_gamma in [0.01, 1, 5]:
#     for this_C in [0.1, 1, 15, 250]:
#         svm = SVC(kernel='rbf', gamma=this_gamma, C=this_C).fit(X_train_scaled, y_train)
#         train_score.append(svm.score(X_train_scaled, y_train))
# train_score

In [ ]:
y_pre_svm = svm.predict(X_test_scaled)
out_svm = pd.DataFrame({'PassengerId': test['PassengerId'].values, 'Survived': y_pre_svm})
# out_svm

In [ ]:
# Naive Bayes Classifier, Highly efficient but bad generalization
nb = GaussianNB().fit(X_train_scaled, y_train)
train_score = nb.score(X_train_scaled, y_train)
train_score  # tend to overfit, because features are many

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3).fit(X_train_scaled, y_train)
y_pre_knn = knn.predict(X_test_scaled)
train_score = knn.score(X_train_scaled, y_train)
train_score

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=4, min_samples_leaf=10).fit(X_train, y_train)
y_pre_tree = tree.predict(X_test)
train_score = tree.score(X_train, y_train)
train_score
# train_score=[]
# for depth in [3, 5, 10, 20, 50]:
#     tree = DecisionTreeClassifier(max_depth=depth).fit(X_train_scaled, y_train)
#     train_score.append(tree.score(X_train_scaled, y_train))
# train_score

In [ ]:
# Neural Networks
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(hidden_layer_sizes=[100, 100], solver='adam', max_iter=300,
                   random_state=1).fit(X_train_scaled, y_train)
y_pre_NN = NN.predict(X_test_scaled)
train_score = NN.score(X_train_scaled, y_train)
train_score


In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=49, max_features='auto', random_state=0).fit(X_train, y_train)
y_pre_rf = rf.predict(X_test)
train_score = rf.score(X_train, y_train)
train_score

In [ ]:
test = pd.read_csv('../input/test.csv')
# select one algorithm according to the performance on trainning set
output = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pre_NN})
output.to_csv('prediction_NN.csv', index=False)
output

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from matplotlib import pyplot as plt


In [ ]:
def split(X, y):
    X_cv, X_dev, y_cv, y_dev = train_test_split(X, y, test_size=0.2, random_state=0)

    return X_cv, X_dev, y_cv, y_dev

In [77]:
def grid_search_cross_validation(X_cv, X_dev, y_cv, y_dev, save_csv=False):
#     model = SVC()
#     parameters = {'C': [2], 'kernel': ['rbf'],
#                   'gamma': ['scale'], 'random_state': [0]}  # larger the gamma, more complex the kernel

    # model = DecisionTreeClassifier()
    # parameters = {'max_depth': [13], 'max_features': [600], 'random_state': [0]}

    # model = LogisticRegression()
    # parameters = {'C': [2], 'solver': ['liblinear'], 'random_state': [0]}

    model = RandomForestClassifier()
    parameters = {'n_estimators': [10, 50, 100, 150], 'max_features': ['auto', 'sqrt', 'log2', None], 
                  'max_depth': [5, 10, 20],
                  'min_samples_leaf': [1, 2, 3],
                  'random_state': [0], 'n_jobs': [8]}

#     model = MLPClassifier()
#     parameters = {'hidden_layer_sizes': [(50, )], 'activation': ['relu'], 'solver': ['adam'],
#                   'max_iter': [1000], 'early_stopping': [False],
#                   'alpha': [0], 'random_state': [0]}

    # model = KNeighborsClassifier()
    # parameters = {'n_neighbors': [10], 'weights': ['distance'], 'n_jobs': [8]}

    clf = GridSearchCV(estimator=model, param_grid=parameters, return_train_score=True, cv=3)
    clf.fit(X_cv, y_cv)

    grid_search_results = pd.DataFrame(clf.cv_results_)
    if save_csv:
        # output grid search results as a csv file
        algorithm = str(model).split('(')[0]
        grid_search_results.to_csv(os.path.join(save_dir, algorithm + ' parameter grid search results.csv'))

    pd.set_option('display.max_columns', None)
    print(grid_search_results)
    print('best score: ' + str(clf.best_score_))
    print('best parameters: ' + str(clf.best_params_))
    
    best_estimator = clf.best_estimator_
    test_score = best_estimator.score(X_dev, y_dev)
    print('best estimator test score: ' + str(test_score))


In [ ]:
def run():
    X = X_train
    y = y_train
    X_cv, X_dev, y_cv, y_dev = split(X, y)
    grid_search_cross_validation(X_cv, X_dev, y_cv, y_dev, save_csv=False)


In [78]:
run()

     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.057635      0.041938         0.105364        0.000091   
1         0.097882      0.003365         0.105319        0.000246   
2         0.145651      0.003907         0.105262        0.000365   
3         0.186912      0.004614         0.105076        0.000286   
4         0.028960      0.000672         0.105121        0.000460   
5         0.096591      0.000952         0.105361        0.000091   
6         0.134992      0.005084         0.105058        0.000216   
7         0.174935      0.008265         0.105082        0.000229   
8         0.029957      0.001071         0.104906        0.000309   
9         0.093398      0.002578         0.105112        0.000212   
10        0.139955      0.007579         0.105133        0.000109   
11        0.181448      0.010597         0.104730        0.000285   
12        0.029931      0.002101         0.104916        0.000076   
13        0.096648      0.002856  

In [81]:
def slected_model_prediction(model):
    y_pre = model.predict(X_test)
    train_score = model.score(X_train, y_train)

    test = pd.read_csv('../input/test.csv')
    # select one algorithm according to the performance on trainning set
    output = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pre})
    output.to_csv('prediction.csv', index=False)
    

In [82]:
model = RandomForestClassifier(max_depth=5, max_features=None, min_samples_leaf=2, n_estimators=150, 
                               n_jobs=8, random_state=0).fit(X_train, y_train)

slected_model_prediction(model)